## Player Rankings and Machine Learning

In [3]:
import pandas as pd
import numpy as np
import os

In [ ]:

path = os.getcwd()

forwards = pd.read_pickle(path+'/preprocessed/final/forwards.pkl')

In [ ]:
pd.set_option('display.max_columns', 999)
forwards.describe().filter(regex='shots')

In [12]:
pd.set_option('display.max_columns', 999)
forwards.describe().filter(regex='shots')

,shots_total_shots,shots_shot_goal,shots_shot_goal %,shots_shot_opportunity,shots_shot_opportunity %,shots_shot_acc,shots_shot_acc %
count,446.000000,446.000000,446.000000,446.000000,446.000000,446.000000,446.000000
mean,37.363229,5.190583,0.124121,28.695067,0.760888,15.441704,0.411457
std,28.347568,5.226295,0.087475,22.536987,0.142887,12.470188,0.138192
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16.000000,1.000000,0.060606,12.000000,0.698316,5.250000,0.344828
50%,32.000000,4.000000,0.120926,24.000000,0.777778,13.000000,0.400000
75%,53.000000,8.000000,0.180118,40.000000,0.846154,22.000000,0.484804
max,167.000000,31.000000,0.600000,133.000000,1.000000,69.000000,1.000000
